In [2]:
from dash import Dash, html, dcc, Input, Output
import dash
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px 
from datetime import timedelta
import glob

In [3]:
file_path = './data/process_data.csv/*.csv'

# Use glob to search for all CSV files in the
fichiers_csv = glob.glob(file_path)

# Create a list to store DataFrames
list_df = []

# Browse each CSV file and read it into a DataFrame
for fichier in fichiers_csv:
  df = pd.read_csv(fichier)
  list_df.append(df)

# Concatenate all DataFrames into one
df = pd.concat(list_df, ignore_index=True)

C:\Users\hilar\AppData\Local\Temp\ipykernel_7392\1577309886.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fichier)
C:\Users\hilar\AppData\Local\Temp\ipykernel_7392\1577309886.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fichier)
C:\Users\hilar\AppData\Local\Temp\ipykernel_7392\1577309886.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fichier)
C:\Users\hilar\AppData\Local\Temp\ipykernel_7392\1577309886.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fichier)
C:\Users\hilar\AppData\Local\Temp\ipykernel_7392\1577309886.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fichier)
C:\Users\hilar\AppData\Lo

In [4]:
df['Trip Start Timestamp'] = pd.to_datetime(df['Trip Start Timestamp'], utc=True).dt.tz_localize(None)

In [5]:
app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=[dbc.themes.BOOTSTRAP])

### tab style
tab_style = {
    'idle':{
        'borderRadius': '10px',
        'padding': '0px',
        'marginInline': '5px',
        'display':'flex',
        'alignItems':'center',
        'justifyContent':'center',
        'fontWeight': 'bold',
        'backgroundColor': '#ffde59',
        'border':'none',
        'color': 'white',
    },
    'active':{
        'borderRadius': '10px',
        'padding': '0px',
        'marginInline': '5px',
        'display':'flex',
        'alignItems':'center',
        'justifyContent':'center',
        'fontWeight': 'bold',
        'border':'none',
        'backgroundColor': '#ff792e',
        'color': 'white',
    }
}

#Filters
date_filter = dcc.DatePickerRange(
    id="slicer-date", 
    min_date_allowed=min(df["Trip Start Timestamp"]),
    max_date_allowed=max(df["Trip Start Timestamp"]),
    start_date=min(df["Trip Start Timestamp"]),
    end_date=min(df["Trip Start Timestamp"]) + timedelta(days=30),
    clearable=False,
    style={'border': 'none', 'borderRadius': '10px', 'margin': '5px'} 
)
payment_type_filter = dcc.Dropdown(
    df['Payment Type'].unique(),
    value=df['Payment Type'].unique()[0],
    placeholder="Payment Type",
    id='slicer-payment-type', className="m-1"
)
period_filter = dcc.Dropdown(
    df['Period'].unique(),
    value=df['Period'].unique()[0],
    placeholder="Period",
    id='slicer-period', className="m-1"
)

app.layout = html.Div([
    dbc.Container(
        fluid=True,
        children=[
            dbc.Row(
                [
                    dbc.NavbarSimple(
                        brand=html.Span("Chicago Taxi Trips Analysis of 2020", style={
                            'color': '#31356e',
                            'font-weight': 'bold',
                            'font-size': '40px',
                            'font-family': 'Schibsted Grotesk'
                        }),
                        brand_href="#",
                        color="#f6b210",
                        dark=True,
                    )
                ],
                align="center",
                className="g-0",
            ),
            dbc.Row([
                dbc.Col(date_filter, width=3, style={'paddingBlock': '10px'}),
                dbc.Col(payment_type_filter, width=3, style={'paddingBlock': '10px'}),
                dbc.Col(period_filter, width=3, style={'paddingBlock': '10px'}),
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Tabs(id='product-tabs', value='tab1', children=[
                        dcc.Tab(label='Geospatial Analysis', value='tab1', style=tab_style['idle'], selected_style=tab_style['active'],
                            children=[
                                html.Div([
                                    dbc.Row([
                                        dbc.Col(
                                            dcc.Dropdown(
                                                id="selected-value",
                                                options=[
                                                    {'label': 'Average Tips', 'value': 'Average_Tips'},
                                                    {'label': 'Average Total', 'value': 'Average_Total'}
                                                ],
                                                value="Average_Total",
                                                clearable=False,
                                            ), width=3, style={'paddingBlock': '10px'},
                                        ),
                                        dbc.Col(
                                            dcc.Dropdown(
                                                id='selected-location',
                                                options=[
                                                    {'label': 'Pickup Location', 'value': 'pickup'},
                                                    {'label': 'Dropoff Location', 'value': 'dropoff'}
                                                ],
                                                value='pickup',
                                                clearable=False,
                                            ), width=3, style={'paddingBlock': '10px'},
                                        ),
                                    ]),
                                    dcc.Graph(id="map"),
                                ], style={'width': '100%', 'marginTop': '15px'}),
                            ]),
                        dcc.Tab(label='Time analysis', value='tab2', style=tab_style['idle'], selected_style=tab_style['active'],
                            children=[
                                html.Div([
                                    dbc.Row([
                                        dbc.Col(
                                            dcc.Dropdown(
                                                id="selected-type",
                                                options=[
                                                    {'label': 'Bar chart', 'value': 'Bar_chart'},
                                                    {'label': 'Line Chart', 'value': 'Line_Chart'}
                                                ],
                                                value="Line_Chart",
                                                clearable=False,
                                                className="m-1"
                                            ), width=3, style={'paddingBlock': '10px'}
                                        ),
                                        dbc.Col(
                                            dcc.Dropdown(
                                                id="selected-value2",
                                                options=[
                                                    {'label': 'Average Trip Duration', 'value': 'Avg_duration'},
                                                    {'label': 'Average Total Fare', 'value': 'Seasonality'}
                                                ],
                                                value="Seasonality",
                                                clearable=False,
                                                className="m-1"
                                            ), width=3, style={'paddingBlock': '10px'}
                                        ),
                                        html.Div([
                                            dbc.Col(dcc.Graph(id='graph',style={'background-color': 'black'})),
                                        ], style={'width': '100%', 'display': 'inline-block' },),
                                    ]),
                                ], style={'width': '100%'}),
                            ]),
                    ], style={'width': '100%','marginTop': '5px','height':'30px'}),
                ], width=10),  

                dbc.Col([
                    dbc.Row(id='card1', style={'width': '100%', 'margin': '10px 0'}),
                    dbc.Row(id='card2', style={'width': '100%', 'margin': '10px 0'}),
                    dbc.Row(id='card3', style={'width': '100%', 'margin': '10px 0'}),
                    dbc.Row(id='card4', style={'width': '100%', 'margin': '10px 0'}),
                ], width=2, style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'flex-end'})
            ])
        ]
    )
], style={'backgroundColor': '#ffffff', 'minHeight': '100vh'})




# Data filter
def filter_data(df, selected_start_date, selected_end_date, selected_payment, selected_period):
    filtered_df = df.copy()

    if selected_start_date:
        filtered_df = filtered_df[filtered_df['Trip Start Timestamp'] >= selected_start_date]
    if selected_end_date:
        filtered_df = filtered_df[filtered_df['Trip Start Timestamp'] <= selected_end_date]
    if selected_payment:
        filtered_df = filtered_df[filtered_df['Payment Type'] == selected_payment]
    if selected_period:
        filtered_df = filtered_df[filtered_df['Period'] == selected_period]

    return filtered_df


def generate_card(title, value):
    return html.Div(
        dbc.Card([
            dbc.CardBody([
                html.P(value, className="card-value",
                       style={'margin': '0px', 'fontSize': '25px', 'fontWeight': 'bold', 'color': '#31356e','font-family': 'Schibsted Grotesk'}),
                html.H4(title, className="card-title",
                        style={'margin': '0px', 'fontSize': '15px', 'color': 'white', 'fontWeight': 'bold'})
            ], style={'textAlign': 'center'}),
        ], style={"backgroundColor": '#f6b210', 'border': 'none', 'borderRadius': '23px', 'margin': '10px'})
    )

# Callback for cards and pie charts
@app.callback(
    [Output('card1', 'children'),
    Output('card2', 'children'),
    Output('card3', 'children'),
    Output('card4', 'children')],
    [Input('slicer-date', 'start_date'),
    Input('slicer-date', 'end_date'),
    Input('slicer-payment-type', 'value'),
    Input('slicer-period', 'value')]
)
def update_cards_and_pie(selected_start_date, selected_end_date, selected_payment, selected_period):
    filtered_df = filter_data(df, selected_start_date, selected_end_date, selected_payment, selected_period)

    card1 = generate_card("Total Trips", f"{len(filtered_df['Trip ID']):,}".replace(',', ' '))
    card2 = generate_card("Sales Total ($)",f"{int(filtered_df['Trip Total'].sum()):,}".replace(',', ' '))
    card3 = generate_card("Km on Average", f"{round(filtered_df['Trip Kilometres'].mean(), 2):,.2f}".replace(',', ' '))
    card4 = generate_card("Minutes on Average", f"{round(filtered_df['Trip Minute'].mean(), 2):,.2f}".replace(',', ' '))
    return card1, card2, card3, card4

import plotly.express as px

@app.callback(
    Output('map', 'figure'),
    [Input('selected-value', 'value'),
     Input('slicer-date', 'start_date'),
     Input('slicer-date', 'end_date'),
     Input('slicer-payment-type', 'value'),
     Input('slicer-period', 'value'),
     Input('selected-location', 'value')]
)
def generate_map(selected_value, selected_start_date, selected_end_date, selected_payment, selected_period, location):
    filtered_df = filter_data(df.copy(), selected_start_date, selected_end_date, selected_payment, selected_period)

    if location == 'pickup':
        lat_col = 'Pickup Centroid Latitude'
        lon_col = 'Pickup Centroid Longitude'
    else:
        lat_col = 'Dropoff Centroid Latitude'
        lon_col = 'Dropoff Centroid Longitude'
    
    if selected_value == 'Average_Tips':
        map_df = filtered_df.groupby([lat_col, lon_col])['Tips'].mean().reset_index()
        map_df.rename(columns={'Tips': 'value'}, inplace=True)
        t = 'Average Tips' 
    else:
        map_df = filtered_df.groupby([lat_col, lon_col])['Trip Total'].mean().reset_index()
        map_df.rename(columns={'Trip Total': 'value'}, inplace=True)
        t = 'Average Total Cost'
    
    map_df['value'] = map_df['value'].round(2)

    fig = px.scatter_mapbox(
        map_df,
        lat=lat_col,
        lon=lon_col,
        hover_data=["value"],
        size="value",
        color="value",
        zoom=10,
    )
    
    fig.update_layout(mapbox_style="open-street-map",
    title = f'{t} based on {location}')
    return fig

@app.callback(
    Output('graph', 'figure'),
    [Input('selected-type', 'value'),
    Input('selected-value2', 'value'),
    Input('slicer-date', 'start_date'),
    Input('slicer-date', 'end_date'),
    Input('slicer-payment-type', 'value'),
    Input('slicer-period', 'value'),
    ]
)
def temporel_analysis(selected_type ,selected_value, selected_start_date, selected_end_date, selected_payment, selected_period):
    filtered_df = filter_data(df.copy(), selected_start_date, selected_end_date, selected_payment, selected_period)

    # Extract date
    filtered_df['Date'] = filtered_df['Trip Start Timestamp'].dt.date

    if selected_value == 'Avg_duration':
        filtered_df = filtered_df.groupby(['Date', 'Period'])['Trip Minute'].mean().reset_index()
        filtered_df.rename(columns={'Trip Minute': 'value'}, inplace=True)
        color ="Period"
    else :
        # Group by date and calculate trip mean
        filtered_df = filtered_df.groupby('Date')['Trip Total'].mean().reset_index()

        # Calculate statistics
        mean_trips = filtered_df['Trip Total'].mean()
        std_trips = filtered_df['Trip Total'].std()

        # Set thresholds for peak and off-peak periods
        peak_threshold = mean_trips + std_trips  # Peak periods
        low_threshold = mean_trips - std_trips   # Low periods

        # Identify peak and low periods
        filtered_df['Seasonality'] = 'Normal'
        filtered_df.loc[filtered_df['Trip Total'] > peak_threshold, 'Seasonality'] = 'Peak'
        filtered_df.loc[filtered_df['Trip Total'] < low_threshold, 'Seasonality'] = 'Low'

        filtered_df.rename(columns={'Trip Total': 'value'}, inplace=True)
        color ="Seasonality"

    # To stop getting a line when no data is available

    # Create a DataFrame of all dates in the interval
    all_dates = pd.date_range(start=selected_start_date, end=selected_end_date).date
    all_dates_df = pd.DataFrame(all_dates, columns=['Date'])

    filtered_df = all_dates_df.merge(filtered_df, on='Date', how='left')
    filtered_df['value'] = filtered_df['value'].where(filtered_df['value'].notnull(), None)
    filtered_df['value'] = filtered_df['value'].round(2)

    if selected_type == "Bar_chart":
        fig = px.bar(
            filtered_df, x="Date", y="value", color=color, barmode="stack", height=430, 
        )
    else:
        if selected_value == 'Avg_duration':
            filtered_df2 = filtered_df.groupby('Date')['value'].mean().reset_index()
            filtered_df2['value'] = filtered_df2['value'].round(2)
            fig = px.line(filtered_df2, x="Date", y="value", height=430, text='value')
        else:
            fig = px.line(filtered_df, x="Date", y="value", height=430, text='value')

    fig.update_layout(
    xaxis_title='',
    yaxis_title=''
)

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)